## 1. Initialization

> a. Read the excel/csv containing the entire query-output

> b. Initialize the parameters of thresholds, directories, and fields to concat for generating individual as well as combined match-score

> c. Get the unique list of countries in present dataframe

In [1]:
import time, numpy as np, pandas as pd, re, string, subprocess #, recordlinkage

_STATIC_FILE_NAME="SM_Temp_Shortlist.xlsx"
_RAW_SCORES_DIRECTORY='Raw_Scores'
_CLEANED_SCORES_DIRECTORY='Cleaned_Scores'
_MASTER_DATA_DIRECTORY='Master_Data'
_FIELDS_TO_CONCAT={
    'CONCAT_ADDRESS':   ['ADDRESS_LINE_1','ADDRESS_LINE_2','ADDRESS_LINE_3']#, 'CONCAT_SRC':       ['SITE_NAME','STATE','CITY','CONCAT_ADDRESS','POSTAL_CODE']
                }

_COLUMNS_TO_CLEAN=['ADDRESS_LINE_1','ADDRESS_LINE_2','ADDRESS_LINE_3','SITE_NAME','STATE','CITY','POSTAL_CODE']
_BINARIES_NAME="levenshtein"
_BINARIES_EXTENSION=".dll"
_MAXSIZE=5000

_THRESHOLD_FOR_INDIVIDUAL=0.85
#_THRESHOLD_FOR_ENTIRE_COMBINED=0.55
_THRESHOLD_FOR_ADDRESS_COMBINED=0.75

_THRESHOLDS_DICT={
    'CONCAT_ADDRESS': _THRESHOLD_FOR_ADDRESS_COMBINED,
    #'CONCAT_SRC': _THRESHOLD_FOR_ENTIRE_COMBINED, 
    'SITE_NAME': _THRESHOLD_FOR_INDIVIDUAL,
    'STATE': _THRESHOLD_FOR_INDIVIDUAL,
    'CITY': _THRESHOLD_FOR_INDIVIDUAL,
    'POSTAL_CODE': _THRESHOLD_FOR_INDIVIDUAL
    }
_COLS_FOR_TOTAL_MATCH_CALC=[colname+'_COMPARISON_SCORE' for colname in _THRESHOLDS_DICT]

_SCALING_FACTOR=3

_TOTAL_MATCHES_THRESHOLD=3


def write_df_to_csv(df, root_dir='', curr_country='', file_suffix='_temp.csv', index_flag=False):
    try:
        abs_path=os.path.join(root_dir, curr_country+file_suffix)
        df.to_csv(abs_path, index=index_flag)
        print(f'\nSuccessfully created \{abs_path}!')
    except:
        print(f'\nSomething went wrong while writing the file. Please check if it is currently in use.')


def preprocess_dataframe(df):
    df.replace(np.nan, '', inplace=True)
    for colname in df.columns.values:
        if colname=='COUNTRY':
            df[colname]=df[colname].apply(lambda x: x.replace(' ','_'))
        df[colname]=df[colname].astype(str).apply(lambda x: x.strip())


def clean_dataframe(df, replace_punctuations=True):
    # Replaced another special character which was causing Italy CSV file read to fail in R
    if replace_punctuations:
        special_chars=re.escape(string.punctuation)+''
        print('\nSpecial Character that will be replaced are:  ', special_chars) # !"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\[\\\\\\]\\{\\|\\}\\~
    for colname in df.columns.values:
        if colname in _COLUMNS_TO_CLEAN and replace_punctuations:
            df[colname]=df[colname].replace(r'['+special_chars+']', '', regex=True).str.lower()
    for colname, cols_to_concat in _FIELDS_TO_CONCAT.items():
        df[colname]=df[cols_to_concat].apply(lambda single_row: ''.join(single_row.values), axis=1)
    df.drop(labels=_FIELDS_TO_CONCAT['CONCAT_ADDRESS'], axis=1, inplace=True)


def scale_up_comparison_score(df, colname='SITE_NAME_COMPARISON_SCORE', scaling_factor=_SCALING_FACTOR):
    print(f'\nScaling up {colname} by {scaling_factor}')
    df[colname]=df[colname].apply(lambda x: x*scaling_factor)


def replace_cyclic_dependencies(df, child_indicator, master_indicator):
    arr=set(df[child_indicator].array)
    for val in df[master_indicator]:
        if val in arr:
            replace_val=df[df[child_indicator]==val][master_indicator].values[0]
            print(val,' found in normalized_duplicates[',child_indicator,']. Replacement: ', replace_val)
            df[master_indicator].replace(val, replace_val, inplace=True)
    return df


def return_top_match(df, child_column, score_key_column):
    normalized_duplicates=df.sort_values(by=[child_column]).sort_values(by=[score_key_column],ascending=False)
    normalized_duplicates=normalized_duplicates.groupby(child_column).head(1).sort_values(by=[child_column])
    return normalized_duplicates


site_master_df=pd.read_excel(_STATIC_FILE_NAME, index_col=0)
preprocess_dataframe(site_master_df)

countries=list(site_master_df['COUNTRY'].unique())
huge_countries=list(site_master_df['COUNTRY'].value_counts()[site_master_df['COUNTRY'].value_counts() > _MAXSIZE].index)
for c in huge_countries:
    countries.remove(c)

print('\nUnique Countries having counts lesser than ', _MAXSIZE, ': ', countries)


Unique Countries having counts lesser than  5000 :  ['United_States']


## 2. Process data in batches: getting scores of all the candidate-pairs is highly computation intensive. RAM crashes for incoming batch-size>4000
> a. Partition the entire input dataset based on country-name. For-loop to do the entire process for all countries.

> b. Create a concatenated (address) field and drop the individual 3 address fields

> c. Create a concatenated (name, address) field

In [2]:
# todo: for loop- Research on multithreading to speed up country-wise batches. RAM might crash for incoming batch-size>4000.

c=0
curr_country=countries[c]
country_df=site_master_df[site_master_df['COUNTRY']==curr_country]
print(curr_country)
clean_dataframe(country_df)
write_df_to_csv(df=country_df[_THRESHOLDS_DICT.keys()], curr_country=curr_country, file_suffix='_country_df.csv', index_flag=True)

# todo: Imputation of address values where Site-name is exactly the same, otherwise it'll result in 2 separate master-records

print(f'\n{curr_country} has {country_df.shape[0]} records')

United_States

Special Character that will be replaced are:   !"\#\$%\&'\(\)\*\+,\-\./:;<=>\?@\[\\\]\^_`\{\|\}\~

Successfully created \United_States_country_df.csv!

United_States has 1000 records


## 3. Initialize the candidate-pairs for comparison via the Index() object, and a Comparer() object with set of fields to compare amongst the candidate pairs

### Currently configured to run this match-score generation in R, such that:

> if individual fields' match-score > 85% , then set the *_col_*_COMPARISON_SCORE column to 1, else 0

> if combined address-fields' match-score > 75% , then set the CONCAT_ADDRESS_COMPARISON_SCORE column to 1, else 0

### Comparer algorithm can be: 'jaro', 'jarowinkler', 'levenshtein', 'damerau_levenshtein', 'qgram', 'cosine', 'smith_waterman', 'lcs'

### You can also set n_job=-1 to use up all cores available for parallel-computation of scores for the candidate-pairs.

In [4]:
# todo: invoke the R-code from Python using 32-bit Rscript 3.4.4 command
from subprocess import Popen, PIPE

pipe = Popen(
    [
    "C:/Program Files/R/R-3.4.4/bin/i386/Rscript", "Site_Master_Record_Linkage.R",
    f"{_BINARIES_NAME} {_BINARIES_EXTENSION} {_THRESHOLD_FOR_INDIVIDUAL} {_THRESHOLD_FOR_ADDRESS_COMBINED} {_SCALING_FACTOR} {curr_country} {_RAW_SCORES_DIRECTORY} {_TOTAL_MATCHES_THRESHOLD} Dedup NA NA"
    ],
    cwd="C:/Users/vdeshpande/Desktop/Del_Project-Takeda/Site_Master_Repo/",
    stdin=PIPE, stdout=PIPE, stderr=PIPE,shell=True
    )

output, error = pipe.communicate()

# Print R-console output
if pipe.returncode == 0:            
    print('R OUTPUT:\n',output.decode())
else:
    print('R OUTPUT:\n',output.decode())
    print('R ERROR:\n',error.decode())

R OUTPUT:
 [1] "levenshtein .dll 0.85 0.75 3 United_States Raw_Scores 3 Dedup NA NA"
[1] "Loading levenshtein.dll !"
         used (Mb) gc trigger (Mb) max used (Mb)
Ncells 120132  3.3     350000  9.4   302969  8.1
Vcells 169833  1.3     786432  6.0   697526  5.4
[1] "NRows= 1000 , Candidate-pairs= 499500 , Columns are "
[1] "SR_NUM"         "CONCAT_ADDRESS" "SITE_NAME"      "STATE"         
[5] "CITY"           "POSTAL_CODE"   
[1] "N_combinations= 499500 , Columns are "
[1] "id1"            "id2"            "CONCAT_ADDRESS" "SITE_NAME"     
[5] "STATE"          "CITY"           "POSTAL_CODE"    "is_match"      
[1] "Scaling up column scores if threshold crossed"
[1] "SITE_NAME  :  0.85"
[1] "STATE  :  0.85"
[1] "CITY  :  0.85"
[1] "POSTAL_CODE  :  0.85"
[1] "CONCAT_ADDRESS  :  0.75"
Time difference of 10.77138 secs
[1] "Raw_Scores//United_States_Score_Features.csv"
[1] "Successfully created //Raw_Scores//United_States_Score_Features.csv !"



## 4. Get the set of potential duplicates where TOTAL_SCORE > THRESHOLD

### Currently configured to perform this in R itself.

> R code will finally generate the *Country*_Score_Features.csv in the /Raw_Scores/ directory

In [5]:
duplicates=pd.read_csv(os.path.join(_RAW_SCORES_DIRECTORY, curr_country+'_Score_Features.csv'))
duplicates['COUNTRY']=curr_country
duplicates.head(30)

,SR_NUM_1,SR_NUM_2,SITE_NAME_COMPARISON_SCORE,STATE_COMPARISON_SCORE,CITY_COMPARISON_SCORE,POSTAL_CODE_COMPARISON_SCORE,CONCAT_ADDRESS_COMPARISON_SCORE,NUM_OF_MATCHES_FOUND,COUNTRY
0,15060,15059,1,1,1,1,3,7,United_States
1,15160,15059,0,1,1,0,3,5,United_States
2,15165,15059,1,1,1,0,0,3,United_States
3,15166,15059,1,1,1,0,3,6,United_States
4,15167,15059,1,1,1,0,0,3,United_States
5,15168,15059,1,1,1,0,3,6,United_States
6,15169,15059,1,1,1,0,0,3,United_States
7,15170,15059,1,1,1,0,3,6,United_States
8,15171,15059,1,1,1,0,3,6,United_States
9,15172,15059,1,1,1,0,3,6,United_States


## 5. Choose the best match for incoming child records based on highest total-score

In [6]:
normalized_duplicates=return_top_match(df=duplicates, child_column='SR_NUM_1', score_key_column='NUM_OF_MATCHES_FOUND')
normalized_duplicates.head(30)

,SR_NUM_1,SR_NUM_2,SITE_NAME_COMPARISON_SCORE,STATE_COMPARISON_SCORE,CITY_COMPARISON_SCORE,POSTAL_CODE_COMPARISON_SCORE,CONCAT_ADDRESS_COMPARISON_SCORE,NUM_OF_MATCHES_FOUND,COUNTRY
0,15060,15059,1,1,1,1,3,7,United_States
33,15062,15061,1,0,1,1,0,3,United_States
34,15070,15066,0,0,0,0,3,3,United_States
41,15071,15070,1,1,1,1,3,7,United_States
42,15072,15070,1,1,1,1,3,7,United_States
74,15073,15072,1,1,1,1,3,7,United_States
60,15074,15071,1,1,1,1,3,7,United_States
76,15075,15072,1,1,1,1,3,7,United_States
104,15076,15074,1,1,1,1,3,7,United_States
63,15077,15071,0,1,0,0,3,4,United_States


## 5. Reusable function to replace the cyclic matches
### For example:

>   Record45 matches with Record44

>   Record67 matches with Record45

### In this case we should maintain:
>   Record67 matches with Record44

In [7]:
normalized_duplicates=replace_cyclic_dependencies(df=normalized_duplicates, child_indicator='SR_NUM_1', master_indicator='SR_NUM_2')
normalized_duplicates

ized_duplicates[ SR_NUM_1 ]. Replacement:  15319
15321  found in normalized_duplicates[ SR_NUM_1 ]. Replacement:  15319
15326  found in normalized_duplicates[ SR_NUM_1 ]. Replacement:  15066
15327  found in normalized_duplicates[ SR_NUM_1 ]. Replacement:  15066
15329  found in normalized_duplicates[ SR_NUM_1 ]. Replacement:  15066
15333  found in normalized_duplicates[ SR_NUM_1 ]. Replacement:  15162
15335  found in normalized_duplicates[ SR_NUM_1 ]. Replacement:  15162
15334  found in normalized_duplicates[ SR_NUM_1 ]. Replacement:  15162
15337  found in normalized_duplicates[ SR_NUM_1 ]. Replacement:  15162
15338  found in normalized_duplicates[ SR_NUM_1 ]. Replacement:  15162
15339  found in normalized_duplicates[ SR_NUM_1 ]. Replacement:  15162
15343  found in normalized_duplicates[ SR_NUM_1 ]. Replacement:  15162
15345  found in normalized_duplicates[ SR_NUM_1 ]. Replacement:  15162
15336  found in normalized_duplicates[ SR_NUM_1 ]. Replacement:  15162
15352  found in normalized_d

,SR_NUM_1,SR_NUM_2,SITE_NAME_COMPARISON_SCORE,STATE_COMPARISON_SCORE,CITY_COMPARISON_SCORE,POSTAL_CODE_COMPARISON_SCORE,CONCAT_ADDRESS_COMPARISON_SCORE,NUM_OF_MATCHES_FOUND,COUNTRY
0,15060,15059,1,1,1,1,3,7,United_States
33,15062,15061,1,0,1,1,0,3,United_States
34,15070,15066,0,0,0,0,3,3,United_States
41,15071,15066,1,1,1,1,3,7,United_States
42,15072,15066,1,1,1,1,3,7,United_States
...,...,...,...,...,...,...,...,...,...
13528,16053,16013,1,1,1,1,3,7,United_States
13551,16055,16054,1,1,1,1,0,4,United_States
13552,16056,16054,1,1,1,1,3,7,United_States
13555,16057,16054,0,0,1,1,3,5,United_States


## 6. CSV for static-analysis of matches

In [8]:
write_df_to_csv(df=normalized_duplicates, root_dir=_CLEANED_SCORES_DIRECTORY, curr_country=curr_country, file_suffix='_Cleaned_Feature_Scores.csv')
print('\n"SR_NUM_2" will be the master record')


Successfully created \Cleaned_Scores\United_States_Cleaned_Feature_Scores.csv!

"SR_NUM_2" will be the master record


## 7. Get the unique set of Master-Records and create a master CSV file for each country

> a. Think of 'SR_NUM_1' as the list of incoming Primary-keys, and 'SR_NUM_2' as the value to which it should be mapped based on match-score

> b. Hence, union of 'SR_NUM_1' & 'SR_NUM_2' will be entire set of duplicates

> c. Stand-alone records in the current country_batch_dataframe will not fall in this entire set of duplicates

> d. Master-records set wil be the sets of 'SR_NUM_2' & #c above.

In [9]:
a1=set(normalized_duplicates['SR_NUM_1'].values.tolist())
a2=set(normalized_duplicates['SR_NUM_2'].values.tolist())
country_set=set(country_df.index.values.tolist())
entire_duplicates_set=a1.union(a2)
no_match_set=country_set.difference(entire_duplicates_set)
master_record_ids=no_match_set.union(a2)



country_df_copy=site_master_df[site_master_df['COUNTRY']==curr_country]
preprocess_dataframe(df=country_df_copy)
clean_dataframe(df=country_df_copy, replace_punctuations=False)

print(f'{site_master_df.shape[0]} records get merged into {len(master_record_ids)}')
#country_master_df=country_df_copy.loc[master_record_ids].drop('CONCAT_SRC',axis=1)
country_master_df=country_df_copy.loc[master_record_ids]
write_df_to_csv(df=country_master_df, root_dir=_MASTER_DATA_DIRECTORY, curr_country=curr_country, index_flag=True, file_suffix='_Master.csv')

1000 records get merged into 157

Successfully created \Master_Data\United_States_Master.csv!


## 8. Get the normalized-duplicates into a CSV to show translation of incoming record into single golden record

> a. Create a master_cross_reference_df for the master_record_ids with relevant scores scaled up by _SCALING_FACTOR, and other comparison scores set to 1

> b. concat it with the normalized_duplicates dataframe

In [10]:
master_record_score_array=[1.0]*len(master_record_ids)
master_record_df_dict={
    'SR_NUM_1': list(master_record_ids),
    'SR_NUM_2': list(master_record_ids),
    'SITE_NAME_COMPARISON_SCORE': master_record_score_array,
    'STATE_COMPARISON_SCORE': master_record_score_array,
    'CITY_COMPARISON_SCORE': master_record_score_array,
    'CONCAT_ADDRESS_COMPARISON_SCORE': master_record_score_array,
    'POSTAL_CODE_COMPARISON_SCORE': master_record_score_array }

cross_ref_df=pd.DataFrame(master_record_df_dict)
cross_ref_df['COUNTRY']=curr_country
scale_up_comparison_score(cross_ref_df,'CONCAT_ADDRESS_COMPARISON_SCORE',_SCALING_FACTOR)
cross_ref_df['NUM_OF_MATCHES_FOUND']=cross_ref_df[_COLS_FOR_TOTAL_MATCH_CALC].sum(axis=1)


cross_ref_df=cross_ref_df.append(normalized_duplicates)
cross_ref_df.sort_values(by=['SR_NUM_1'], axis=0, inplace=True)

write_df_to_csv(df=cross_ref_df, root_dir=_MASTER_DATA_DIRECTORY, curr_country=curr_country, file_suffix='_Raw_Cross_Ref.csv')


Scaling up CONCAT_ADDRESS_COMPARISON_SCORE by 3

Successfully created \Master_Data\United_States_Raw_Cross_Ref.csv!


## 9. Generate the report to display name & address fields of match-and-merge combinations

> a. Merge the master_cross_reference_df with the country_batch_dataframe as a left-outer-join on Primary-key='SR_NUM_1'

> b. Merge this master_cross_reference_df with the country_batch_dataframe as a left-outer-join on Primary-key='SR_NUM_2'

In [11]:
country_df_copy.reset_index(inplace=True)
country_df_colnames=country_df_copy.columns.values

country_df_copy.columns=[colname+'_1' for colname in country_df_colnames]
cross_ref_df=cross_ref_df.merge(country_df_copy, how='left', on='SR_NUM_1')

country_df_copy.columns=[colname+'_2' for colname in country_df_colnames]
cross_ref_df=cross_ref_df.merge(country_df_copy, how='left', on='SR_NUM_2')
cross_ref_df=cross_ref_df[['SR_NUM_1', 'SR_NUM_2', 'SITE_NAME_1','SITE_NAME_2','SITE_NAME_COMPARISON_SCORE','STATE_1','STATE_2','STATE_COMPARISON_SCORE', 'CITY_1','CITY_2','CITY_COMPARISON_SCORE','CONCAT_ADDRESS_1','CONCAT_ADDRESS_2','CONCAT_ADDRESS_COMPARISON_SCORE', 'POSTAL_CODE_1','POSTAL_CODE_2','POSTAL_CODE_COMPARISON_SCORE','NUM_OF_MATCHES_FOUND']]

write_df_to_csv(df=cross_ref_df, root_dir=_MASTER_DATA_DIRECTORY, curr_country=curr_country, file_suffix='_Cross_Ref_Full_Report.csv')


Successfully created \Master_Data\United_States_Cross_Ref_Full_Report.csv!
